In [2]:
import sys
sys.path.insert(0, "..")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math

In [3]:
!ls -lah analysis/not_uploaded/IA/ | grep regress

-rw-r--r-- 1 janpf janpf 902M Nov  6 21:07 .scratch.ckpts.IA.pexels.scores-None.change_regress.epoch-8.pth.txt
-rw-r--r-- 1 janpf janpf 952M Nov  6 21:07 .scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-17.pth.txt
-rw-r--r-- 1 janpf janpf 936M Nov  6 21:07 .scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-5.pth.txt
-rw-r--r-- 1 janpf janpf 935M Nov  6 21:07 .scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-9.pth.txt


In [4]:
df = pd.read_csv("analysis/not_uploaded/IA/.scratch.ckpts.IA.pexels.scores-one.epoch-13.pth.txt")

for col in df.columns:
    if not "score" in col:
        continue

    df[col] = df[col].apply(lambda row: eval(row)[0])


df["intensity"] = [float("nan")]*len(df)
df.loc[df["level"] == 0, "intensity"] = 0
for t in ["styles", "technical", "composition"]:
    for dist in mapping[t].keys():
        for pol in mapping[t][dist].keys():
            if pol == "neg":
                for lvl, change in enumerate(reversed(mapping[t][dist][pol])):
                    df.loc[(df["distortion"] == change.split(";")[0])&(df["level"] == float(change.split(";")[1])), "intensity"] = -(lvl+1)
            if pol == "pos":
                for lvl, change in enumerate(mapping[t][dist][pol]):
                    df.loc[(df["distortion"] == change.split(";")[0])&(df["level"] == float(change.split(";")[1])), "intensity"] = lvl+1
df["intensity"] = df["intensity"].astype(int)

df

In [26]:
orig_best = 0
overall = 0


original_df = df[df["distortion"] == "original"]
if distortion in parameter_range:
    original_df["level"] = parameter_range[distortion]["default"]

parameter_df = df[df["distortion"] == distortion]
corr_df = pd.concat([parameter_df, original_df])[["img", "distortion", "level", "score"]]

if distortion in parameter_range:
    default = parameter_range[distortion]["default"]
else:
    default = 0

if polarity == "pos":
    corr_df = corr_df[corr_df["level"] >= default]
else:
    corr_df = corr_df[corr_df["level"] <= default]

corr_df["level"] = corr_df["level"].apply(lambda x: abs((x) - (default)))

for i, f in enumerate(df["img"].unique()):
    corr_df_img = corr_df[corr_df["img"] == f]
    
    max_val = float(max(corr_df_img["score"]))
    ori_val = float(corr_df_img[corr_df_img["distortion"] == "original"].iloc[0]["score"])
    
    overall +=1
    if math.isclose(max_val,ori_val):
        orig_best +=1

orig_best, overall

(6802, 14999)

In [27]:
orig_best / overall

0.45349689979331953

In [28]:
overall/6

2499.8333333333335

In [29]:
1/6

0.16666666666666666